In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent.parent))
%config Completer.use_jedi = False

## Librerias

In [2]:
# random search forecaster
# ==============================================================================
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from skforecast.model_selection.model_selection import random_search_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom

In [17]:
from sklearn.model_selection import ParameterSampler
from scipy.stats.distributions import expon

rng = np.random.RandomState(0)
param_grid = {'a':[1, 2], 'b': expon()}
# param_grid = {'a':[1, 2], 'b': [3, 4]}
param_list = list(ParameterSampler(param_grid, n_iter=4,
                                 random_state=rng))

print(param_list)

rounded_list = [dict((k, round(v, 6)) for (k, v) in d.items())
                for d in param_list]


rounded_list == [{'b': 0.89856, 'a': 1},
                 {'b': 0.923223, 'a': 1},
                 {'b': 1.878964, 'a': 2},
                 {'b': 1.038159, 'a': 2}]

[{'a': 1, 'b': 0.8985603929935616}, {'a': 1, 'b': 0.9232231458040688}, {'a': 2, 'b': 1.8789640641973517}, {'a': 2, 'b': 1.0381592949436094}]


True

In [15]:
from sklearn.model_selection import ParameterSampler
from scipy.stats.distributions import expon

rng = np.random.RandomState(0)
param_grid = {'a':[1, 2], 'b': expon()}
param_grid = {'a':[1, 2], 'b': [3, 4]}
param_list = ParameterSampler(param_grid, n_iter=4,
                                 random_state=rng)

param_list = list(param_list)
print(param_list)

keys = param_list[0].keys()
print(keys)

{'b': [3,4]}

dic_final = {}

for key in keys:
    values = []
    for dic in param_list:
        values.append(dic[key])
        
    dic_final[key] = list(set(values))

print("")
print(dic_final)



# b = set(val for dic in a for val in dic.values())
# b

[{'b': 3, 'a': 1}, {'b': 4, 'a': 1}, {'b': 3, 'a': 2}, {'b': 4, 'a': 2}]
dict_keys(['b', 'a'])

{'b': [3, 4], 'a': [1, 2]}


In [16]:
from sklearn.model_selection import ParameterSampler
from scipy.stats.distributions import expon

rng = np.random.RandomState(0)

param_distributions = {'a':[1, 2], 'b': expon()}
n_iter = 4

param_distributions = list(ParameterSampler(param_distributions, n_iter=n_iter, random_state=rng))

param_grid = {}

for key in param_distributions[0].keys():
    values = []
    for dic in param_distributions:
        values.append(dic[key])
        
    param_grid[key] = list(set(values))

print("")
print(param_grid)


{'a': [1, 2], 'b': [0.8985603929935616, 0.9232231458040688, 1.0381592949436094, 1.8789640641973517]}


In [2]:
param_grid = {'a':[1, 2], 'b': range(10)}

In [4]:
param_list = list(ParameterSampler(param_grid, n_iter=4,
                                   random_state=0))

param_list

[{'b': 8, 'a': 2}, {'b': 1, 'a': 1}, {'b': 9, 'a': 2}, {'b': 8, 'a': 1}]

## Random search forecaster

In [3]:
# Download data
# ==============================================================================
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv')
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y/%m/%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]
data = data.sort_index()

# Train-val-test dates
# ==============================================================================
end_train = '2001-01-01 23:59:00'
end_val = '2006-01-01 23:59:00'

print(f"Train dates      : {data.index.min()} --- {data.loc[:end_train].index.max()}  (n={len(data.loc[:end_train])})")
print(f"Validation dates : {data.loc[end_train:].index.min()} --- {data.loc[:end_val].index.max()}  (n={len(data.loc[end_train:end_val])})")
print(f"Test dates       : {data.loc[end_val:].index.min()} --- {data.index.max()}  (n={len(data.loc[end_val:])})")


Train dates      : 1991-07-01 00:00:00 --- 2001-01-01 00:00:00  (n=115)
Validation dates : 2001-02-01 00:00:00 --- 2006-01-01 00:00:00  (n=60)
Test dates       : 2006-02-01 00:00:00 --- 2008-06-01 00:00:00  (n=29)


In [5]:
# Random search hyperparameter and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags      = 10 # Placeholder, the value will be overwritten
             )

# Lags used as predictors
lags_grid = [3, 5]

# Regressor hyperparameters
param_distributions = {'n_estimators': np.arange(start=10, stop=100, step=1, dtype=int),
                       'max_depth': np.arange(start=5, stop=30, step=1, dtype=int)}

results_random = random_search_forecaster(
                        forecaster  = forecaster,
                        y           = data.loc[:end_val, 'y'],
                        steps       = 12,
                        lags_grid   = lags_grid,
                        param_distributions  = param_distributions,
                        n_iter      = 5,
                        metric      = 'mean_squared_error',
                        refit       = True,
                        initial_train_size = len(data.loc[:end_train]),
                        fixed_train_size   = False,
                        return_best = True,
                        random_state = 12,
                        verbose     = True
                 )

results_random

Number of models compared: 10


loop param_grid:   0%|                                              | 0/5 [00:00<?, ?it/s]

Information of backtesting process
----------------------------------
Number of observations used for initial training or as initial window: 115
Number of observations used for backtesting: 60
    Number of folds: 6
    Number of steps per fold: 11
    Last fold only includes 5 observations

Data partition in fold: 0
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2001-02-01 00:00:00 -- 2001-12-01 00:00:00  (n=11)
Data partition in fold: 1
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2002-01-01 00:00:00 -- 2002-11-01 00:00:00  (n=11)
Data partition in fold: 2
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2002-12-01 00:00:00 -- 2003-10-01 00:00:00  (n=11)
Data partition in fold: 3
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2003-11-01 00:00:00 -- 2004-09-01 00:00:00  (n=11)
Data partition in fold: 4
    Training:   1991-07-01 00:00:00 -


loop param_grid:  20%|███████▌                              | 1/5 [00:00<00:01,  2.20it/s]

Information of backtesting process
----------------------------------
Number of observations used for initial training or as initial window: 115
Number of observations used for backtesting: 60
    Number of folds: 6
    Number of steps per fold: 11
    Last fold only includes 5 observations

Data partition in fold: 0
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2001-02-01 00:00:00 -- 2001-12-01 00:00:00  (n=11)
Data partition in fold: 1
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2002-01-01 00:00:00 -- 2002-11-01 00:00:00  (n=11)
Data partition in fold: 2
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2002-12-01 00:00:00 -- 2003-10-01 00:00:00  (n=11)
Data partition in fold: 3
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2003-11-01 00:00:00 -- 2004-09-01 00:00:00  (n=11)
Data partition in fold: 4
    Training:   1991-07-01 00:00:00 -

...


loop param_grid:  80%|██████████████████████████████▍       | 4/5 [00:01<00:00,  2.81it/s]

Information of backtesting process
----------------------------------
Number of observations used for initial training or as initial window: 115
Number of observations used for backtesting: 60
    Number of folds: 6
    Number of steps per fold: 11
    Last fold only includes 5 observations

Data partition in fold: 0
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2001-02-01 00:00:00 -- 2001-12-01 00:00:00  (n=11)
Data partition in fold: 1
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2002-01-01 00:00:00 -- 2002-11-01 00:00:00  (n=11)
Data partition in fold: 2
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2002-12-01 00:00:00 -- 2003-10-01 00:00:00  (n=11)
Data partition in fold: 3
    Training:   1991-07-01 00:00:00 -- 2001-01-01 00:00:00  (n=115)
    Validation: 2003-11-01 00:00:00 -- 2004-09-01 00:00:00  (n=11)
Data partition in fold: 4
    Training:   1991-07-01 00:00:00 -


loop lags_grid: 100%|███████████████████████████████████████| 2/2 [00:03<00:00,  1.73s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3] 
  Parameters: {'n_estimators': 77, 'max_depth': 25}
  Backtesting metric: 0.06590698227625151



,lags,params,metric,n_estimators,max_depth
0,"[1, 2, 3]","{'n_estimators': 77, 'max_depth': 25}",0.065907,77,25
2,"[1, 2, 3]","{'n_estimators': 74, 'max_depth': 20}",0.072082,74,20
8,"[1, 2, 3, 4, 5]","{'n_estimators': 33, 'max_depth': 19}",0.073652,33,19
3,"[1, 2, 3]","{'n_estimators': 33, 'max_depth': 19}",0.074437,33,19
5,"[1, 2, 3, 4, 5]","{'n_estimators': 77, 'max_depth': 25}",0.075426,77,25
1,"[1, 2, 3]","{'n_estimators': 81, 'max_depth': 23}",0.075779,81,23
9,"[1, 2, 3, 4, 5]","{'n_estimators': 17, 'max_depth': 27}",0.080211,17,27
6,"[1, 2, 3, 4, 5]","{'n_estimators': 81, 'max_depth': 23}",0.082055,81,23
7,"[1, 2, 3, 4, 5]","{'n_estimators': 74, 'max_depth': 20}",0.082560,74,20
4,"[1, 2, 3]","{'n_estimators': 17, 'max_depth': 27}",0.084555,17,27


In [5]:
# Random search hyperparameter and lags
# ==============================================================================
def create_predictors(y):
    '''
    Create first 5 lags of a time series.
    '''
    
    lags = y[-1:-6:-1]
    
    return lags 

forecaster = ForecasterAutoregCustom(
                    regressor      = RandomForestRegressor(random_state=123),
                    fun_predictors = create_predictors,
                    window_size    = 5
                )

# Lags used as predictors
lags_grid = [3, 5]

# Regressor hyperparameters
param_distributions = {'n_estimators': np.arange(start=10, stop=100, step=1, dtype=int),
                       'max_depth': np.arange(start=5, stop=30, step=1, dtype=int)}

results_random = random_search_forecaster(
                        forecaster  = forecaster,
                        y           = data.loc[:end_val, 'y'],
                        steps       = 12,
                        lags_grid   = lags_grid,
                        param_distributions  = param_distributions,
                        n_iter      = 5,
                        metric      = 'mean_squared_error',
                        refit       = True,
                        initial_train_size = len(data.loc[:end_train]),
                        fixed_train_size   = False,
                        return_best = True,
                        random_state = 123,
                        verbose     = False
                 )

results_random

/home/javi/Documents/GitHub/skforecast/skforecast/model_selection/model_selection.py:1280: UserWarning: `lags_grid` ignored if forecaster is an instance of `ForecasterAutoregCustom`.
  warnings.warn(


Number of models compared: 5


loop lags_grid: 100%|███████████████████████████████████████| 1/1 [00:04<00:00,  4.10s/it]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: custom predictors 
  Parameters: {'n_estimators': 77, 'max_depth': 17}
  Backtesting metric: 0.03147248676391345



,lags,params,metric,n_estimators,max_depth
4,custom predictors,"{'n_estimators': 77, 'max_depth': 17}",0.031472,77,17
2,custom predictors,"{'n_estimators': 66, 'max_depth': 24}",0.033616,66,24
0,custom predictors,"{'n_estimators': 96, 'max_depth': 19}",0.033761,96,19
1,custom predictors,"{'n_estimators': 52, 'max_depth': 17}",0.040640,52,17
3,custom predictors,"{'n_estimators': 94, 'max_depth': 28}",0.044071,94,28


## test mocked 0.4.3

In [1]:
# random search forecaster
# ==============================================================================
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

from skforecast.model_selection.model_selection import grid_search_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom

import warnings
from tqdm import tqdm
from functools import partialmethod

In [2]:
y = pd.Series(
        np.array([0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
                  0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
                  0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
                  0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
                  0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
                  0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
                  0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
                  0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
                  0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
                  0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453]))

In [3]:
# Grid search hyperparameter and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                regressor = Ridge(random_state=123),
                lags      = 2 # Placeholder, the value will be overwritten
             )

steps = 3
n_validation = 12
y_train = y[:-n_validation]
lags_grid = [2, 4]
param_grid = {'alpha': [0.01, 0.1, 1]}

results = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = y,
                        lags_grid   = lags_grid,
                        param_grid  = param_grid,
                        steps       = steps,
                        refit       = False,
                        metric      = 'mean_squared_error',
                        initial_train_size = len(y_train),
                        fixed_train_size   = False,
                        return_best = True,
                        verbose     = False
               )
display(results)

display(results.lags.values)
display(results.params.values)
display(results.metric.values)
display(results.alpha.values)

Number of models compared: 6


loop param_grid:   0%|                                              | 0/3 [00:00<?, ?it/s]
                                                                                          
loop lags_grid: 100%|███████████████████████████████████████| 2/2 [00:00<00:00, 38.92it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2] 
  Parameters: {'alpha': 0.01}
  Backtesting metric: 0.06464646415433532



,lags,params,metric,alpha
0,"[1, 2]",{'alpha': 0.01},0.064646,0.01
1,"[1, 2]",{'alpha': 0.1},0.065024,0.10
2,"[1, 2]",{'alpha': 1},0.067455,1.00
3,"[1, 2, 3, 4]",{'alpha': 0.01},0.067793,0.01
4,"[1, 2, 3, 4]",{'alpha': 0.1},0.068025,0.10
5,"[1, 2, 3, 4]",{'alpha': 1},0.069486,1.00


array([array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2, 3, 4]),
       array([1, 2, 3, 4]), array([1, 2, 3, 4])], dtype=object)

array([{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}, {'alpha': 0.01},
       {'alpha': 0.1}, {'alpha': 1}], dtype=object)

array([0.06464646, 0.06502362, 0.06745534, 0.06779272, 0.06802481,
       0.06948609])

array([0.01, 0.1 , 1.  , 0.01, 0.1 , 1.  ])

In [4]:
# Grid search hyperparameter and lags
# ==============================================================================
def create_predictors(y):
    '''
    Create first 4 lags of a time series.
    '''
    
    lags = y[-1:-5:-1]
    
    return lags 

forecaster = ForecasterAutoregCustom(
                    regressor      = Ridge(random_state=123),
                    fun_predictors = create_predictors,
                    window_size    = 4
                )

steps = 3
n_validation = 12
y_train = y[:-n_validation]
param_grid = {'alpha': [0.01, 0.1, 1]}

results = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = y,
                        lags_grid   = lags_grid,
                        param_grid  = param_grid,
                        steps       = steps,
                        refit       = False,
                        metric      = 'mean_squared_error',
                        initial_train_size = len(y_train),
                        fixed_train_size   = False,
                        return_best = True,
                        verbose     = False
               )
display(results)

display(results.lags.values)
display(results.params.values)
display(results.metric.values)
display(results.alpha.values)

/home/javi/anaconda3/envs/skforecast/lib/python3.8/site-packages/skforecast/model_selection/model_selection.py:1085: UserWarning: `lags_grid` ignored if forecaster is an instance of `ForecasterAutoregCustom`.
  warnings.warn(


Number of models compared: 3


loop lags_grid: 100%|███████████████████████████████████████| 1/1 [00:00<00:00, 31.41it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: custom predictors 
  Parameters: {'alpha': 0.01}
  Backtesting metric: 0.06779271738965444



,lags,params,metric,alpha
0,custom predictors,{'alpha': 0.01},0.067793,0.01
1,custom predictors,{'alpha': 0.1},0.068025,0.10
2,custom predictors,{'alpha': 1},0.069486,1.00


array(['custom predictors', 'custom predictors', 'custom predictors'],
      dtype=object)

array([{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}], dtype=object)

array([0.06779272, 0.06802481, 0.06948609])

array([0.01, 0.1 , 1.  ])

In [37]:
from sklearn.model_selection import ParameterGrid

param_grid = {'alpha': [0.01, 0.1, 1]}
list(ParameterGrid(param_grid))

[{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}]

In [35]:
# test
# ==============================================================================
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.model_selection.model_selection import grid_search_forecaster

from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True) # hide progress bar


def test_output_evaluate_grid_hyperparameters_ForecasterAutoreg_with_mocked():
    '''
    Test output of _evaluate_grid_hyperparameters in ForecasterAutoreg with mocked
    (mocked done in Skforecast v0.4.3)
    '''
    forecaster = ForecasterAutoreg(
                    regressor = Ridge(random_state=123),
                    lags      = 2 # Placeholder, the value will be overwritten
                 )

    steps = 3
    n_validation = 12
    y_train = y[:-n_validation]
    lags_grid = [2, 4]
    param_grid = {'alpha': [0.01, 0.1, 1]}
    idx = len(lags_grid)*len(param_grid['alpha'])

    results = grid_search_forecaster(
                            forecaster  = forecaster,
                            y           = y,
                            lags_grid   = lags_grid,
                            param_grid  = param_grid,
                            steps       = steps,
                            refit       = False,
                            metric      = 'mean_squared_error',
                            initial_train_size = len(y_train),
                            fixed_train_size   = False,
                            return_best = False,
                            verbose     = False
                   )
    
    expected_results = pd.DataFrame({
            'lags'  :[[1, 2], [1, 2], [1, 2], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]],
            'params':[{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}, {'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}],
            'metric':np.array([0.06464646, 0.06502362, 0.06745534, 0.06779272, 0.06802481, 0.06948609]),                                                               
            'alpha' :np.array([0.01, 0.1 , 1.  , 0.01, 0.1 , 1.  ])
                                     },
            index=np.arange(idx)
                                   )

    pd.testing.assert_frame_equal(results, expected_results)
    
    
def test_output_evaluate_grid_hyperparameters_ForecasterAutoregCustom_with_mocked():
    '''
    Test output of _evaluate_grid_hyperparameters in ForecasterAutoregCustom with mocked
    (mocked done in Skforecast v0.4.3)
    '''
    def create_predictors(y):
        '''
        Create first 4 lags of a time series, used in ForecasterAutoregCustom.
        '''

        lags = y[-1:-5:-1]

        return lags
    
    forecaster = ForecasterAutoregCustom(
                        regressor      = Ridge(random_state=123),
                        fun_predictors = create_predictors,
                        window_size    = 4
                    )

    steps = 3
    n_validation = 12
    y_train = y[:-n_validation]
    param_grid = {'alpha': [0.01, 0.1, 1]}
    idx = len(param_grid['alpha'])

    results = grid_search_forecaster(
                            forecaster  = forecaster,
                            y           = y,
                            param_grid  = param_grid,
                            steps       = steps,
                            refit       = False,
                            metric      = 'mean_squared_error',
                            initial_train_size = len(y_train),
                            fixed_train_size   = False,
                            return_best = False,
                            verbose     = False
                   )
    
    expected_results = pd.DataFrame({
            'lags'  :['custom predictors', 'custom predictors', 'custom predictors'],
            'params':[{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}],
            'metric':np.array([0.06779272, 0.06802481, 0.06948609]),                                                               
            'alpha' :np.array([0.01, 0.1 , 1.])
                                     },
            index=np.arange(idx)
                                   )
    
    pd.testing.assert_frame_equal(results, expected_results)
    

def test_evaluate_grid_hyperparameters_when_return_best():
    '''
    Test forecaster is refited when return_best=True in _evaluate_grid_hyperparameters
    '''
    forecaster = ForecasterAutoreg(
                    regressor = Ridge(random_state=123),
                    lags      = 2 # Placeholder, the value will be overwritten
                 )

    steps = 3
    n_validation = 12
    y_train = y[:-n_validation]
    lags_grid = [2, 4]
    param_grid = {'alpha': [0.01, 0.1, 1]}    

    grid_search_forecaster(
                            forecaster  = forecaster,
                            y           = y,
                            lags_grid   = lags_grid,
                            param_grid  = param_grid,
                            steps       = steps,
                            refit       = False,
                            metric      = 'mean_squared_error',
                            initial_train_size = len(y_train),
                            fixed_train_size   = False,
                            return_best = True,
                            verbose     = False
                   )
    
    expected_lags = np.array([1, 2])
    expected_alpha = 0.01
    
    assert (expected_lags == forecaster.lags).all()
    assert expected_alpha == forecaster.regressor.alpha

In [36]:
test_output_evaluate_grid_hyperparameters_ForecasterAutoreg_with_mocked()
test_output_evaluate_grid_hyperparameters_ForecasterAutoregCustom_with_mocked()
test_evaluate_grid_hyperparameters_when_return_best()

Number of models compared: 6
Number of models compared: 3
Number of models compared: 6
`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2] 
  Parameters: {'alpha': 0.01}
  Backtesting metric: 0.06464646415433532



## test new functions 0.5.0

### _evaluate_grid_hyperparameters

In [ ]:
# Unit test _evaluate_grid_hyperparameters
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.model_selection.model_selection import _evaluate_grid_hyperparameters

from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True) # hide progress bar

# Fixtures _backtesting_forecaster_refit Series (skforecast==0.4.2)
# np.random.seed(123)
# y = np.random.rand(50)

y = pd.Series(
    np.array([0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
              0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
              0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
              0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
              0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
              0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
              0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
              0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
              0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
              0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453]))


def test_output_evaluate_grid_hyperparameters_ForecasterAutoreg_with_mocked():
    '''
    Test output of _evaluate_grid_hyperparameters in ForecasterAutoreg with mocked
    (mocked done in Skforecast v0.4.3)
    '''
    forecaster = ForecasterAutoreg(
                    regressor = Ridge(random_state=123),
                    lags      = 2 # Placeholder, the value will be overwritten
                 )

    steps = 3
    n_validation = 12
    y_train = y[:-n_validation]
    lags_grid = [2, 4]
    param_grid = [{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}]
    idx = len(lags_grid)*len(param_grid)

    results = _evaluate_grid_hyperparameters(
                            forecaster  = forecaster,
                            y           = y,
                            lags_grid   = lags_grid,
                            param_grid  = param_grid,
                            steps       = steps,
                            refit       = False,
                            metric      = 'mean_squared_error',
                            initial_train_size = len(y_train),
                            fixed_train_size   = False,
                            return_best = False,
                            verbose     = False
                             )
    
    expected_results = pd.DataFrame({
            'lags'  :[[1, 2], [1, 2], [1, 2], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]],
            'params':[{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}, {'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}],
            'metric':np.array([0.06464646, 0.06502362, 0.06745534, 0.06779272, 0.06802481, 0.06948609]),                                                               
            'alpha' :np.array([0.01, 0.1 , 1.  , 0.01, 0.1 , 1.  ])
                                     },
            index=np.arange(idx)
                                   )

    pd.testing.assert_frame_equal(results, expected_results)
    
    
def test_output_evaluate_grid_hyperparameters_ForecasterAutoregCustom_with_mocked():
    '''
    Test output of _evaluate_grid_hyperparameters in ForecasterAutoregCustom with mocked
    (mocked done in Skforecast v0.4.3)
    '''
    def create_predictors(y):
        '''
        Create first 4 lags of a time series, used in ForecasterAutoregCustom.
        '''

        lags = y[-1:-5:-1]

        return lags
    
    forecaster = ForecasterAutoregCustom(
                        regressor      = Ridge(random_state=123),
                        fun_predictors = create_predictors,
                        window_size    = 4
                    )

    steps = 3
    n_validation = 12
    y_train = y[:-n_validation]
    param_grid = [{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}]
    idx = len(param_grid)

    results = _evaluate_grid_hyperparameters(
                            forecaster  = forecaster,
                            y           = y,
                            param_grid  = param_grid,
                            steps       = steps,
                            refit       = False,
                            metric      = 'mean_squared_error',
                            initial_train_size = len(y_train),
                            fixed_train_size   = False,
                            return_best = False,
                            verbose     = False
                            )
    
    expected_results = pd.DataFrame({
            'lags'  :['custom predictors', 'custom predictors', 'custom predictors'],
            'params':[{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}],
            'metric':np.array([0.06779272, 0.06802481, 0.06948609]),                                                               
            'alpha' :np.array([0.01, 0.1 , 1.])
                                     },
            index=np.arange(idx)
                                   )
    
    pd.testing.assert_frame_equal(results, expected_results)
    

def test_evaluate_grid_hyperparameters_when_return_best():
    '''
    Test forecaster is refited when return_best=True in _evaluate_grid_hyperparameters
    '''
    forecaster = ForecasterAutoreg(
                    regressor = Ridge(random_state=123),
                    lags      = 2 # Placeholder, the value will be overwritten
                 )

    steps = 3
    n_validation = 12
    y_train = y[:-n_validation]
    lags_grid = [2, 4]
    param_grid = [{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}]

    _evaluate_grid_hyperparameters(
            forecaster  = forecaster,
            y           = y,
            lags_grid   = lags_grid,
            param_grid  = param_grid,
            steps       = steps,
            refit       = False,
            metric      = 'mean_squared_error',
            initial_train_size = len(y_train),
            fixed_train_size   = False,
            return_best = True,
            verbose     = False
            )
    
    expected_lags = np.array([1, 2])
    expected_alpha = 0.01
    
    assert (expected_lags == forecaster.lags).all()
    assert expected_alpha == forecaster.regressor.alpha

### grid_search_forecaster

In [2]:
from sklearn.model_selection import ParameterGrid

param_grid = {'alpha': [0.01, 0.1, 1]}
list(ParameterGrid(param_grid))

[{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}]

In [12]:
# Unit test grid_search_forecaster
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster

from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True) # hide progress bar

# Fixtures _backtesting_forecaster_refit Series (skforecast==0.4.2)
# np.random.seed(123)
# y = np.random.rand(50)

y = pd.Series(
    np.array([0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
              0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
              0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
              0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
              0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
              0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
              0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
              0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
              0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
              0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453]))

def test_output_grid_search_forecaster_ForecasterAutoreg_with_mocked():
    '''
    Test output of grid_search_forecaster in ForecasterAutoreg with mocked
    (mocked done in Skforecast v0.4.3)
    '''
    forecaster = ForecasterAutoreg(
                    regressor = Ridge(random_state=123),
                    lags      = 2 # Placeholder, the value will be overwritten
                 )

    steps = 3
    n_validation = 12
    y_train = y[:-n_validation]
    lags_grid = [2, 4]
    param_grid = {'alpha': [0.01, 0.1, 1]}
    idx = len(lags_grid)*len(param_grid['alpha'])

    results = grid_search_forecaster(
                            forecaster  = forecaster,
                            y           = y,
                            lags_grid   = lags_grid,
                            param_grid  = param_grid,
                            steps       = steps,
                            refit       = False,
                            metric      = 'mean_squared_error',
                            initial_train_size = len(y_train),
                            fixed_train_size   = False,
                            return_best = False,
                            verbose     = False
                            )
    
    expected_results = pd.DataFrame({
            'lags'  :[[1, 2], [1, 2], [1, 2], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]],
            'params':[{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}, {'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}],
            'metric':np.array([0.06464646, 0.06502362, 0.06745534, 0.06779272, 0.06802481, 0.06948609]),                                                               
            'alpha' :np.array([0.01, 0.1 , 1.  , 0.01, 0.1 , 1.  ])
                                     },
            index=np.arange(idx)
                                   )

    pd.testing.assert_frame_equal(results, expected_results)

In [13]:
test_output_grid_search_forecaster_ForecasterAutoreg_with_mocked()

Number of models compared: 6


### random_search_forecaster

In [9]:
from sklearn.model_selection import ParameterSampler


param_grid = {'alpha':np.logspace(-5, 3, 10)}
param_list = list(ParameterSampler(param_grid, n_iter=3, random_state=123))

print(param_list)

[{'alpha': 0.03593813663804626}, {'alpha': 1e-05}, {'alpha': 16.681005372000556}]


In [11]:
# Grid search hyperparameter and lags mocked random search
# ==============================================================================
forecaster = ForecasterAutoreg(
                regressor = Ridge(random_state=123),
                lags      = 2 # Placeholder, the value will be overwritten
             )

steps = 3
n_validation = 12
y_train = y[:-n_validation]
lags_grid = [2, 4]
param_grid = {'alpha': [0.03593813663804626, 1e-05, 16.681005372000556]}

results = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = y,
                        lags_grid   = lags_grid,
                        param_grid  = param_grid,
                        steps       = steps,
                        refit       = False,
                        metric      = 'mean_squared_error',
                        initial_train_size = len(y_train),
                        fixed_train_size   = False,
                        return_best = False,
                        verbose     = False
                        )
display(results)

display(results.lags.values)
display(results.params.values)
display(results.metric.values)
display(results.alpha.values)

Number of models compared: 6


,lags,params,metric,alpha
1,"[1, 2]",{'alpha': 1e-05},0.064602,0.000010
0,"[1, 2]",{'alpha': 0.03593813663804626},0.064759,0.035938
4,"[1, 2, 3, 4]",{'alpha': 1e-05},0.067766,0.000010
3,"[1, 2, 3, 4]",{'alpha': 0.03593813663804626},0.067861,0.035938
5,"[1, 2, 3, 4]",{'alpha': 16.681005372000556},0.071348,16.681005
2,"[1, 2]",{'alpha': 16.681005372000556},0.071610,16.681005


array([array([1, 2]), array([1, 2]), array([1, 2, 3, 4]),
       array([1, 2, 3, 4]), array([1, 2, 3, 4]), array([1, 2])],
      dtype=object)

array([{'alpha': 1e-05}, {'alpha': 0.03593813663804626}, {'alpha': 1e-05},
       {'alpha': 0.03593813663804626}, {'alpha': 16.681005372000556},
       {'alpha': 16.681005372000556}], dtype=object)

array([0.06460234, 0.06475887, 0.06776596, 0.06786132, 0.0713478 ,
       0.07161   ])

array([1.00000000e-05, 3.59381366e-02, 1.00000000e-05, 3.59381366e-02,
       1.66810054e+01, 1.66810054e+01])

In [20]:
# Unit test random_search_forecaster
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import random_search_forecaster

from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True) # hide progress bar

# Fixtures _backtesting_forecaster_refit Series (skforecast==0.4.2)
# np.random.seed(123)
# y = np.random.rand(50)

y = pd.Series(
    np.array([0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
              0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
              0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
              0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
              0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
              0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
              0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
              0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
              0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
              0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453]))

def test_output_random_search_forecaster_ForecasterAutoreg_with_mocked():
    '''
    Test output of random_search_forecaster in ForecasterAutoreg with mocked
    (mocked done in Skforecast v0.4.3)
    '''
    forecaster = ForecasterAutoreg(
                    regressor = Ridge(random_state=123),
                    lags      = 2 # Placeholder, the value will be overwritten
                 )

    steps = 3
    n_validation = 12
    y_train = y[:-n_validation]
    lags_grid = [2, 4]
    param_distributions = {'alpha':np.logspace(-5, 3, 10)}
    n_iter = 3

    results = random_search_forecaster(
                        forecaster   = forecaster,
                        y            = y,
                        lags_grid    = lags_grid,
                        param_distributions  = param_distributions,
                        steps        = steps,
                        refit        = False,
                        metric       = 'mean_squared_error',
                        initial_train_size = len(y_train),
                        fixed_train_size   = False,
                        n_iter       = n_iter,
                        random_state = 123,
                        return_best  = False,
                        verbose      = False
                        )
    
    expected_results = pd.DataFrame({
            'lags'  :[[1, 2], [1, 2], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2]],
            'params':[{'alpha': 1e-05}, {'alpha': 0.03593813663804626}, {'alpha': 1e-05},
                      {'alpha': 0.03593813663804626}, {'alpha': 16.681005372000556}, {'alpha': 16.681005372000556}],
            'metric':np.array([0.06460234, 0.06475887, 0.06776596, 
                               0.06786132, 0.0713478, 0.07161]),                                                               
            'alpha' :np.array([1.00000000e-05, 3.59381366e-02, 1.00000000e-05, 
                               3.59381366e-02, 1.66810054e+01, 1.66810054e+01])
                                     },
            index=[1, 0, 4, 3, 5, 2]
                                   )

    pd.testing.assert_frame_equal(results, expected_results)

In [21]:
test_output_random_search_forecaster_ForecasterAutoreg_with_mocked()

Number of models compared: 6
